In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 19/02/2025.
@author: Air.Zou
"""
# ... existing code ...
import tushare as ts
import pandas as pd
# 设置 tushare token
ts.set_token('2876ea85cb005fb5fa17c809a98174f2d5aae8b1f830110a5ead6211')
pro = ts.pro_api()

df = pro.fund_daily(ts_code='510310.SH', start_date='20241010', end_date='20250217')
print(df.head())


     ts_code trade_date  pre_close   open   high    low  close  change  \
0  510310.SH   20250217      3.877  3.890  3.892  3.855  3.886   0.009   
1  510310.SH   20250214      3.847  3.840  3.882  3.840  3.877   0.030   
2  510310.SH   20250213      3.855  3.852  3.875  3.841  3.847  -0.008   
3  510310.SH   20250212      3.819  3.818  3.861  3.803  3.855   0.036   
4  510310.SH   20250211      3.835  3.839  3.841  3.807  3.819  -0.016   

   pct_chg         vol       amount  
0   0.2321  5059698.89  1959512.306  
1   0.7798  3506845.82  1355264.298  
2  -0.2075  4126833.85  1590991.272  
3   0.9427  2841055.19  1087206.279  
4  -0.4172  3677965.61  1404633.404  


In [2]:
df["date"]=pd.to_datetime(df["trade_date"])
df['openinterest']=0
df.rename(columns={'amount':'volume'}, inplace=True)
df.set_index("date",inplace=True)
df = df[["open","high","low","close","volume","openinterest"]]
print(df.head())


             open   high    low  close       volume  openinterest
date                                                             
2025-02-17  3.890  3.892  3.855  3.886  1959512.306             0
2025-02-14  3.840  3.882  3.840  3.877  1355264.298             0
2025-02-13  3.852  3.875  3.841  3.847  1590991.272             0
2025-02-12  3.818  3.861  3.803  3.855  1087206.279             0
2025-02-11  3.839  3.841  3.807  3.819  1404633.404             0


In [3]:

import matplotlib
import pandas as pd
import talib
from datetime import datetime
import backtrader as bt
import matplotlib.pyplot as plt
import akshare as ak
import numpy as np
class MyStrategy(bt.Strategy):
    def __init__(self):
        # 添加 MA5 和 MA10 指标
        self.sma5 = bt.indicators.SimpleMovingAverage(self.data.close, period=3)
        self.sma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=20)
        # 计算 MA5 和 MA10 的交叉信号
        self.crossover = bt.indicators.CrossOver(self.sma5, self.sma10)

    def next(self):
        # 如果 MA5 上穿 MA10，买入
        if self.crossover > 0:
            print("买入")
            self.buy()
        # 如果 MA5 下穿 MA10，卖出
        elif self.crossover < 0:
            print("卖出")
            self.sell()


In [4]:
import backtrader as bt

class DynamicPositionStrategy(bt.Strategy):
    def __init__(self):
        self.sma5 = bt.indicators.SimpleMovingAverage(self.data.close, period=3)
        self.sma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.crossover = bt.indicators.CrossOver(self.sma5, self.sma10)
        # 假设使用波动率指标来动态调整仓位
        self.volatility = bt.indicators.StdDev(self.data.close, period=20)

    def next(self):
        if self.crossover > 0:
            available_cash = self.broker.get_cash()
            # 手动计算波动率的均值
            volatility_data = self.volatility.get(size=20)  # 获取最近 20 个周期的波动率数据
            if len(volatility_data) == 20:  # 确保有足够的数据来计算均值
                volatility_mean = sum(volatility_data) / len(volatility_data)
            else:
                # 如果数据不足，使用默认值，这里简单设为 0
                volatility_mean = 0
            # 根据波动率动态调整仓位比例
            if self.volatility[0] > volatility_mean:
                # 高波动率时使用 30% 的资金
                invest_ratio = 0.3
            else:
                # 低波动率时使用 70% 的资金
                invest_ratio = 0.7
            invest_amount = available_cash * invest_ratio
            price = self.data.close[0]
            size = invest_amount / price
            self.buy(size=size)
            print('buy size:', size)
        elif self.crossover < 0:
            available_cash = self.broker.get_cash()
            self.sell(size=self.position.size)
            price = self.data.close[0]
            # 原代码中这里计算卖出数量有误，应该是卖出持仓数量
            # size = available_cash * price  修改为
            size = self.position.size
            print('sell size', size)


In [5]:
import backtrader as bt
import pandas as pd
from datetime import datetime

class DynamicPositionStrategy2(bt.Strategy):
    def __init__(self):
        self.sma5 = bt.indicators.SimpleMovingAverage(self.data.close, period=3)
        self.sma10 = bt.indicators.SimpleMovingAverage(self.data.close, period=10)
        self.crossover = bt.indicators.CrossOver(self.sma5, self.sma10)
        # 假设使用波动率指标来动态调整仓位
        self.volatility = bt.indicators.StdDev(self.data.close, period=20)

    def next(self):
        # 获取当前日期
        current_date = self.data.datetime.date(0)
        # 获取当前可用资金
        available_cash = self.broker.get_cash()
        # 获取当前持仓数量
        position_size = self.position.size
        # 获取当前价格
        price = self.data.close[0]

        # 手动计算波动率的均值
        volatility_data = self.volatility.get(size=20)  # 获取最近 20 个周期的波动率数据
        if len(volatility_data) == 20:  # 确保有足够的数据来计算均值
            volatility_mean = sum(volatility_data) / len(volatility_data)
        else:
            # 如果数据不足，使用默认值，这里简单设为 0
            volatility_mean = 0

        if self.crossover > 0:
            # 计算可投资金比例
            if self.volatility[0] > volatility_mean:
                # 高波动率时使用 30% 的资金
                invest_ratio = 0.3
            else:
                # 低波动率时使用 70% 的资金
                invest_ratio = 0.7
            invest_amount = available_cash * invest_ratio

            if invest_amount > 0:
                # 计算可买入数量
                size = invest_amount / price
                self.buy(size=size)
                print(f'{current_date}: 买入信号触发，可用资金: {available_cash:.2f}，投资比例: {invest_ratio * 100:.2f}%，买入数量: {size:.2f}')
            else:
                print(f'{current_date}: 买入信号触发，但可用资金不足')

        elif self.crossover < 0:
            if position_size > 0:
                self.sell(size=position_size)
                print(f'{current_date}: 卖出信号触发，卖出数量: {position_size:.2f}')
            else:
                print(f'{current_date}: 卖出信号触发，但没有持仓')


In [6]:

# 创建 Cerebro 引擎
cerebro = bt.Cerebro()

# 添加策略
cerebro.addstrategy(DynamicPositionStrategy2)

s = datetime.strptime("20241010", "%Y%m%d")
e = datetime.strptime("20250217", "%Y%m%d")
cerebro.adddata(bt.feeds.PandasData(dataname=df,fromdate=s,todate=e))

# 设置初始资金
cerebro.broker.set_cash(1000000.0)

# 设置仓位


# 设置交易手续费
cerebro.broker.setcommission(commission=0.002)

# 运行回测
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
print('最终资金: %.2f' % cerebro.broker.getvalue())
# 绘制结果
cerebro.plot(style='candlestick', iplot=False, block=True)

Starting Portfolio Value: 1000000.00
2025-01-06: 买入信号触发，可用资金: 1000000.00，投资比例: 70.00%，买入数量: 189189.19
2024-12-19: 卖出信号触发，但没有持仓
2024-12-12: 买入信号触发，可用资金: 1000000.00，投资比例: 30.00%，买入数量: 75930.14
2024-12-05: 卖出信号触发，但没有持仓
2024-11-20: 买入信号触发，可用资金: 1000000.00，投资比例: 70.00%，买入数量: 179211.47
2024-11-04: 卖出信号触发，但没有持仓
2024-10-24: 买入信号触发，可用资金: 1000000.00，投资比例: 30.00%，买入数量: 77942.32
2024-10-17: 卖出信号触发，但没有持仓
2024-10-14: 买入信号触发，可用资金: 1000000.00，投资比例: 70.00%，买入数量: 176723.05
Final Portfolio Value: 1000000.00
最终资金: 1000000.00


2025-02-19 13:55:48.013 python[77273:6734231] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-19 13:55:48.013 python[77273:6734231] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-02-19 13:55:48.023 python[77273:6734231] error messaging the mach port for IMKCFRunLoopWakeUpReliable


[[<Figure size 1280x960 with 6 Axes>]]